Tout d'abord, nous importons les bibliothèques permettant de récupérer les données. La bliothèque requests permet d'envoyer des requêtes à des sites web, bs4 permet d'analyser et extraire des données de documents HTML et donc de sites internet, re permet de rechercher des expressions régulières (regex) dans du texte, pandas permet de manipuler des bases de données et rapidfuzz fournit des outils pour comparer des chaînes de caractères.

In [2]:
import requests
import bs4
import re
import pandas as pd
from rapidfuzz import fuzz

# Requirements : pip install bs4 lxml rapidfuzz

On cherche tout d'abord à confectionner une table avec les informations sur le livre et le numéro d'index auquel il correspond.


In [3]:
# URL du fichier d'index des textes
url_liste_textes = "https://www.gutenberg.org/dirs/GUTINDEX.ALL.iso-8859-1.txt"

# Téléchargement du fichier d'index
request_liste_textes = requests.get(url_liste_textes).content
page = bs4.BeautifulSoup(request_liste_textes, "lxml")
body = page.find("body")
index_texte = body.get_text()